In [1]:
import xml.etree.ElementTree as ET

data = '''
<person>
  <name>Chuck</name>
  <phone type="intl">
     +1 734 303 4456
   </phone>
   <email hide="yes"/>
</person>'''

tree = ET.fromstring(data)
print('Name:', tree.find('name').text)
print('Attr:', tree.find('email').get('hide'))

# Code: http://www.py4e.com/code3/xml1.py
# Or select Download from this trinket's left-hand menu

Name: Chuck
Attr: yes


In [6]:
import urllib.request, urllib.parse, urllib.error
import json
import ssl

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'
# https://developers.google.com/maps/documentation/geocoding/intro

if api_key is False:
    api_key = 42
    serviceurl = 'http://py4e-data.dr-chuck.net/json?'
else :
    serviceurl = 'https://maps.googleapis.com/maps/api/geocode/json?'

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

while True:
    address = input('Enter location: ')
    if len(address) < 1: break

    parms = dict()
    parms['address'] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters')

    try:
        js = json.loads(data)
    except:
        js = None

    if not js or 'status' not in js or js['status'] != 'OK':
        print('==== Failure To Retrieve ====')
        print(data)
        continue

    print(json.dumps(js, indent=4))

    lat = js['results'][0]['geometry']['location']['lat']
    lng = js['results'][0]['geometry']['location']['lng']
    print('lat', lat, 'lng', lng)
    location = js['results'][0]['formatted_address']
    print(location)

Enter location: Atlantic Ocean
Retrieving http://py4e-data.dr-chuck.net/json?address=Atlantic+Ocean&key=42
Retrieved 1228 characters
{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Atlantic Ocean",
                    "short_name": "Atlantic Ocean",
                    "types": [
                        "establishment",
                        "natural_feature"
                    ]
                }
            ],
            "formatted_address": "Atlantic Ocean",
            "geometry": {
                "bounds": {
                    "northeast": {
                        "lat": 68.1962901,
                        "lng": 19.9970779
                    },
                    "southwest": {
                        "lat": -70.7734622,
                        "lng": -82.9931606
                    }
                },
                "location": {
                    "lat": -14.5994134,
                    "lng": -28.

## Exercise 1

Change either the [geojson](www.py4e.com/code3/geojson.py) or [geoxml](www.py4e.com/code3/geoxml.py) to print out the two-character country code from the retrieved data. Add error checking so your program does not traceback if the country code is not there. Once you have it working, search for "Atlantic Ocean" and make sure it can handle locations that are not in any country.

In [177]:
api_key = False

# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'
# https://developers.google.com/maps/documentation/geocoding/intro

if api_key is False:
    api_key = 42
    serviceurl = 'http://py4e-data.dr-chuck.net/json?'
else :
    serviceurl = 'https://maps.googleapis.com/maps/api/geocode/json?'

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

def country_code(address,debug=False):
    if len(address) < 1: 
        print('Empty Query')
        return

    parms = dict()
    parms['address'] = address

    if api_key is not False: parms['key'] = api_key
    
    url = serviceurl + urllib.parse.urlencode(parms)

    #print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    #print('Retrieved', len(data), 'characters')

    try:
        js = json.loads(data)
    except:
        js = None

    if not js or 'status' not in js or js['status'] != 'OK':
        print('==== Failure To Retrieve ====')
        if debug: print(data)
    else:
        if debug: print(json.dumps(js, indent=3))
        _x = None
        for comp in js['results'][0]['address_components']:
            if 'country' in comp['types']:
                _x =comp['short_name']
                print(comp['short_name'],js['results'][0]['formatted_address'])
        if 'establishment' in js['results'][0]['types'] and _x is None:
            print(address+' is a {0} and does not have a country code'.format(js['results'][0]['types'][-1]))
            return js
        if _x is None:
            print(address+' is a {0} and does not have a country code'.format(js['results'][0]['types'][0]))
    return js

In [180]:
for i in ['France','Rome','Poggio Perugino','Nissoria','contigliano','Rieti'
         'Brazil','Tokyo','Total','Via Crati 10','Rocky Mountain','White House',
         'Antartica','Arctic','Total','Enna']:
    data =country_code(i)

FR France
IT Rome, Metropolitan City of Rome, Italy
IT 02100 Poggio Perugino RI, Italy
IT 94010 Nissoria, Free municipal consortium of Enna, Italy
IT 02043 Contigliano, Province of Rieti, Italy
IT 02100 Rieti, Province of Rieti, Italy
JP Tokyo, Japan
==== Failure To Retrieve ====
IT Via Crati, 10, 00199 Roma RM, Italy
Rocky Mountain is a natural_feature and does not have a country code
US 1600 Pennsylvania Avenue NW, Washington, DC 20500, USA
AQ Antarctica
Arctic is a colloquial_area and does not have a country code
==== Failure To Retrieve ====
IT 94100 Enna, Free municipal consortium of Enna, Italy
